# ffmpeg常用用法

## 下载安装

[下载](https://www.ffmpeg.org/download.html)可执行文件包。可直接执行，自主决定是否加入系统环境变量

* 选装CUDA驱动

```
ffmpeg -hwaccels # 检查ffmpeg是否支持gpu加速

ffmpeg -hwaccel cuvid [-hwaccel_device 0] paras # 用法
# 示例
ffmpeg -hwaccel cuvid -hwaccel_device 0 -c:v h264_cuvid -i <input> -c:v h264_nvenc -b:v 2048k -vf scale_npp=1280:-1 -y <output>
```

## 常用命令

### 查看视频文件信息

```
ffmpeg -i vedio.mp4
ffprobe vedio.mp4

```

### 视频转码为h264编码的视频-CPU

如不指定，自动识别视频的编解码器

```
ffmpeg -i <input> [-c:v libx264] -b:v 2048k -vf scale=1280:-1 -y <output>
    -c:v：指定编码器，编码器列表可以使用ffmpeg -codecs查看
    -vf scale：指定输出视频的宽高，高-1代表按照比例自动适应
    -b:v：指定输出视频的码率，即输出视频每秒的bit数
    libx264支持的其他参数请使用ffmpeg -h encoder=libx264命令查询，如转码为其他编码，也可使用类似命令查询可用参数
```

### 视频转码为h264编码的视频-GPU

必须指定编解码器，用ffprobe识别视频的编码格式，*指定对应的GPU解码器*

```
ffmpeg -hwaccel cuvid -c:v h264_cuvid -i <input> -c:v h264_nvenc -b:v 2048k -vf scale_npp=1280:-1 -y <output>
    -hwaccel cuvid：指定使用cuvid硬件加速
    -c:v h264_cuvid：使用h264_cuvid进行视频解码
    -c:v h264_nvenc：使用h264_nvenc进行视频编码
    -b:v：指定输出视频的码率，即输出视频每秒的bit数
    -vf scale_npp=1280:-1：指定输出视频的宽高，注意，这里和软解码时使用的-vf scale=x:x不一样
```

## Map

The -map option is used to choose which streams from the input(s) should be included in the output(s). The -map option can also be used to exclude specific streams with negative mapping.

### Default behavior

If you do not use the -map option then the default stream selection behavior will automatically choose streams.
* Default stream selection will not automatically choose all of the streams.
* Only one stream per type will be selected. For example, if the input has 3 video streams it will only choose 1.
* The default stream selection will choose streams based upon specific criteria.

Using the **-map** option disables the default stream selection behavior and allows you to manually choose streams.

### Syntax

The syntax is: **-map input_file_index:stream_type_specifier:stream_index**

* input_file_index refers to an input and by default will include all of its streams.
* stream_type_specifier (optional) is a stream specifier which will limit the selection to a specific stream type. Stream specifiers * include video, audio, subtitles, data, attachments. Most of the examples below use stream specifiers.
* stream_index (optional) will select a specific stream. If stream_type_specifier is used then the stream_index only refers to the selected stream type and ignores all other stream types.

**Note:** Index numbers start from 0. So the first input or stream is #0, second input or stream is #1, third input or stream is #2, etc.

### Order

The order of -map options is important:

* -map order determines the stream order in the output(s).
* Mapping is applied in order. This is useful when using negative mapping.

### Simple examples

To better understand the syntax here are some examples and descriptions:

* -map 0 From input index #0 (the 1st input) select all streams.
* -map 1:a From input index #1 (the 2nd input) select all audio streams.
* -map 3:s:4 From input index #3 (the 4th input) select subtitle stream index #4 (the fifth subtitle stream).
* -map 0 -map -0:s Will select all streams from input index #0 (the 1st input) except subtitles. The - indicates negative mapping.

![map](ffmpeg-map.png)

## 音视频流操作

```
# 分离音视频
ffmpeg -i input.mp4 -c copy -map 0:v:0 vedio_only.mp4
ffmpeg -i input.mp4 -c copy -map 0:a:0 audio_only.aac

# 合并音视频
ffmpeg -i input.mp4 -i audio.m4a -c copy output.mp4
ffmpeg -i input.mp4 -i audio.m4a -map 0:v:0 -map 1:a:0 -c copy output.mp4
ffmpeg -i input.mp4 -i mandarin.m4a -c copy -map 0:v:0 -map 0:a:0 -map 1:a:0 output.mp4

# 重排音视频
# what if the input file has audio as stream #0 and video as stream #1
ffmpeg -i input.mp4 -map 0:v -map 0:a -c copy output.mp4
```

## 压制字幕

```bash
# 提取字幕
ffmpeg -i subtitle.srt subtitle.ass
ffmpeg -i subtitle.ass subtitle.srt
# 硬字幕嵌入到视频
ffmpeg -i input.mp4 -vf subtitles=subtitle.srt output.mp4
ffmpeg -i input.mp4 -vf subtitles=input.mp4 output.mp4
# 将input.mkv中的第 2 个字幕嵌入
ffmpeg -i input.mp4 -vf subtitles=input.mkv:si=1 output.mp4
# 设置视频、声音、字幕参数 
# -c copy, -c:v copy 复制视频，-c:a aac(指定编码)|copy， -c:s mov_text(保留字幕流)|copy
ffmpeg -i input.mp4 -i subtitles.srt -c:s mov_text -c:v copy -c:a copy output.mp4
# 多字幕、音轨压制
ffmpeg -i input.mp4 -i mandarin.m4a -i english.m4a -i chs.srt -i eng.srt -c copy -map 0:v:0 -map 1:a:0 -map 2:a:0 -map -c:s mov_text -metadata:s:s:0 language=chs -metadata:s:s:1 language=eng output.mp4
# 定制一个带字幕的mp4
ffmpeg -i input.mkv  -map 0:0 -map 0:2 -c:a aac  -vf subtitles=input.mkv output.mp4
```

## 播放速率

```
# 倍速音频
ffmpeg -i input.mkv -filter:a "atempo = 2.0[,atempo=2.0]" -vn output.mkv
    atempo = 2.0，调整音频的采样率为原来的2倍，参数范围0.5~2.0
    
# 倍速视频
ffmpeg -i input.mkv [-r 120] -filter:v "setpts=0.25*PTS" output.mkv
    # -r 120，表示输出视频fps
    # setpts=0.25*PTS，表示帧间隔为输入视频的0.25倍，即4倍速

# 同时调整音视频
ffmpeg -i input.mkv -filter_complex "[0:v]setpts=0.5*PTS[v];[0:a]atempo=2.0[a]" -map "[v]" -map "[a]" output.mkv
```

```bash
# 指定分辨率
ffmpeg -i "5G+AGV.20210624.mp4" -s 1280x720 1.mp4
# 指定码率
ffmpeg -i "5G+AGV.20210624.mp4" -b:v 1.5M 1.mp4
```

## 参考

[使用FFmpeg将字幕文件集成到视频文件](https://www.yaosansi.com/post/ffmpeg-burn-subtitles-into-video/)

[wiki/Map](http://trac.ffmpeg.org/wiki/Map)
